# Julia wrapper

This is a short example on how one could interface with the mobius dlls from the Julia language.

In [1]:
using Libdl

using Plots

plotly()


# How to build the wrapper:

simplyp = dlopen("../Applications/SimplyP/simplyp.so") #.dll on windows

setup_model_h   = dlsym(simplyp, "DllSetupModel")
run_model_h     = dlsym(simplyp, "DllRunModel")
get_timesteps_h = dlsym(simplyp, "DllGetTimesteps")
get_result_h    = dlsym(simplyp, "DllGetResultSeries")

function setup_model(parfile, inputfile)
    return ccall(setup_model_h, Ptr{Cvoid}, (Cstring, Cstring), parfile, inputfile)
end

function run_model(dataset)
    ccall(run_model_h, Cvoid, (Ptr{Cvoid},), dataset)
end

function get_timesteps(dataset)
    return timesteps = ccall(get_timesteps_h, UInt64, (Ptr{Cvoid},), dataset)
end

function get_result_series(dataset, name, indexes)
    timesteps = get_timesteps(dataset)
    result = Array{Float64, 1}(undef, timesteps)
    ccall(get_result_h, Cvoid, (Ptr{Cvoid}, Cstring, Ptr{Cstring}, UInt64, Ptr{Float64}), dataset, name, indexes, length(indexes), result)
    return result
end


# Usage code:

dataset = setup_model("../Applications/SimplyP/Tarland/TarlandParameters_v0-3.dat", "../Applications/SimplyP/Tarland/TarlandInputs.dat")

run_model(dataset)

flow = get_result_series(dataset, "Reach flow (daily mean, cumecs)", ["Coull"])

timesteps = get_timesteps(dataset)

dlclose(simplyp) #only do this after you are done...

Plots.plot(1:timesteps, flow)

<!DOCTYPE html>
 
 
 Plots.jl